In [10]:
import pandas as pd
import re
import sys
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tag import UnigramTagger
from nltk.corpus import brown
import webcolors
sys.path.append("..")
from ADA_JEX2017.Project.Functions.str_functions import *
from ADA_JEX2017.Project.Functions.pre_process import *


In [11]:
path='../ADA_JEX2017/Project/Functions/'

In [12]:
#our datafile exported to csv
data_file='./recipeInfo/recipeInfo_WestWhiteHorvitz_WWW2013.csv'
raw_data = pd.read_csv(data_file ,sep=';')

In [13]:
with open(path+'units_list.txt', 'r') as f:
    units_list = [line.rstrip('\n') for line in f]
    
with open(path+'technique_list.txt', 'r') as f:
    techniques_list = [line.rstrip('\n') for line in f]

with open(path+'ingredient_list.txt', 'r') as f:
    ingredient_list = [line.rstrip('\n') for line in f]
    
lemmatizer = WordNetLemmatizer()
ingredient_list=[lemmatizer.lemmatize(token).lower() for token in ingredient_list]

In [14]:
ingr_dataframe=raw_data[['title','ingredients_list','ingredients_bag-of-words']].copy().dropna()
ingr_dataframe = ingr_dataframe.reset_index(drop=True)
display(ingr_dataframe.head())
ingr_data_reduced=ingr_dataframe.head(100)
#we ignore upper case in the ingredients string
ingr_dataframe['ingredients_list']=ingr_dataframe['ingredients_list'].str.lower()

,title,ingredients_list,ingredients_bag-of-words
0,Easy Light Chocolate Milkshake Recipe,"put one half cup of milk, 4 tablespoons of cho...","! , . 4 ? and are blend chocolate cup enjoy ex..."
1,Lamb Stew Recipe : : Recipes : Food Network,5 yellow onions|2 turnips|5 carrots|1 stalk fe...,", 1 1\/2 1\/4 1\/8 1Â 1\/2 2 3 5 and anise app..."
2,Chocolate Bread Pudding Recipe : Paula Deen : ...,"1 (1-pound) loaf French or Italian bread, cube...",", -lrb- -rrb- 1 1-pound 1\/2 1\/4 1Â 1\/2 2 3 ..."
3,Snowball Cookies II Recipe,1/2 cup powdered sugar|1/3 cup butter or marga...,1 1\/2 1\/3 2 almond baking bisquick butter ch...
4,Old Fashioned Butterscotch Pie Recipe #31698,1 1/2 cup brown sugar|1 cup water|3 eggs|4 tab...,1 1\/2 1\/4 1Â 1\/2 3 4 brown butter cream cup...


In [15]:
add_preprocess(ingr_dataframe,units_list)

In [16]:
ingr_data_reduced=ingr_dataframe.head(100)
all_dic=[]
i=0
for index, row in ingr_dataframe.iterrows():
    recipe=row['Recipe_preporcess']
    dic_ingre,dictec=extract_ingredients\
            (recipe,ingredient_list,techniques_list,units_list)
    all_dic.append(dic_ingre)

In [17]:
ingredients_frame=pd.DataFrame(data=all_dic)
display(ingredients_frame.head(5))

,acidulated water,acorn squash,ale,aleppo pepper,allspice,almond,almond extract,almond milk,amaranth,amaretti,...,wild rice,wine,worcestershire sauce,wrap,yam,yeast,yellow lentil,yoghurt,yuzu,zest
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/8 cup
2,NaN,NaN,NaN,NaN,NaN,NaN,2 teaspoon,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,1 teaspoon,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
print('There are : ',len(list(ingredients_frame)), 'ingredients')
ingred_used={}
for i in list(ingredients_frame):
    ingred_used[i]=sum(ingredients_frame[i].value_counts())

There are :  789 ingredients


In [19]:
occu=sorted((value,key) for (key,value) in ingred_used.items())
occu[::-1]

[(20021, 'salt'),
 (14108, 'onion'),
 (13826, 'sugar'),
 (12737, 'butter'),
 (12001, 'flour'),
 (11280, 'egg'),
 (10467, 'pepper'),
 (9825, 'garlic'),
 (9651, 'water'),
 (8790, 'black pepper'),
 (8706, 'olive oil'),
 (7074, 'milk'),
 (5692, 'tomato'),
 (5065, 'cream'),
 (4871, 'brown sugar'),
 (4728, 'vanilla extract'),
 (4611, 'vegetable oil'),
 (4370, 'cinnamon'),
 (4361, 'clove'),
 (4211, 'parsley'),
 (3816, 'lemon juice'),
 (3798, 'sauce'),
 (3602, 'baking powder'),
 (3531, 'broth'),
 (3301, 'soda'),
 (3276, 'carrot'),
 (3068, 'celery'),
 (2809, 'potato'),
 (2758, 'cheese'),
 (2525, 'cream cheese'),
 (2513, 'chicken breast'),
 (2503, 'basil'),
 (2425, 'parmesan cheese'),
 (2415, 'thyme'),
 (2410, 'oregano'),
 (2320, 'vinegar'),
 (2278, 'cheddar cheese'),
 (2265, 'ginger'),
 (2215, 'beef'),
 (2191, 'margarine'),
 (2177, 'oil'),
 (2157, 'bread'),
 (2132, 'mushroom'),
 (2063, 'nutmeg'),
 (2008, 'soy sauce'),
 (1938, 'bean'),
 (1937, 'cumin'),
 (1933, 'vanilla'),
 (1928, 'seasoning'),
